In [1]:
# import cudf
import numpy as np
import pandas as pd
from PIL import Image
import cv2
import os
import tqdm
import xgboost as xgb
import time
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, fbeta_score
import torch
from torch import nn, optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, random_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
from scipy.stats import randint, uniform
from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils.class_weight import compute_sample_weight
import warnings
import pickle
warnings.filterwarnings("ignore")

In [2]:
# resize images
def resize_image_in_folder(input_dir, output_dir, size=(224, 224), desc='resizing images'):
    if not os.path.exists(input_dir):
        print(f"Input directory {input_dir} does not exist. Please check the path.")
        return

    os.makedirs(output_dir, exist_ok=True)
    supported_formats = ('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff', '.webp')
    for filename in os.listdir(input_dir):
        if filename.lower().endswith(supported_formats):
            img_input_path = os.path.join(input_dir, filename)
            img_output_path = os.path.join(output_dir, filename)
            try:
                img = cv2.imread(img_input_path, cv2.IMREAD_UNCHANGED)

                if img is None:
                    print(f"Error loading {img_input_path}")
                    continue
                resized_img = cv2.resize(img, size, interpolation=cv2.INTER_LANCZOS4)

                if img_output_path.lower().endswith(('.jpg', '.jpeg')) and resized_img.shape[-1] == 4:
                    resized_img = cv2.cvtColor(resized_img, cv2.COLOR_BGRA2BGR)
                cv2.imwrite(img_output_path, resized_img)
            except Exception as e:
                print(f"Error processing {img_input_path}: {e}")
# process all folders
def batch_resize_images(base_input_dir, base_output_dir, size=(128, 128)):
    if not os.path.exists(base_input_dir):
        print(f"Base directory {base_input_dir} does not exist. Please check the path.")
        return

    os.makedirs(base_output_dir, exist_ok=True) # if output directory does not exist, create it.

    for folder in tqdm.tqdm(os.listdir(base_input_dir)):
        current_input_subfolder = os.path.join(base_input_dir, folder)
        current_output_subfolder = os.path.join(base_output_dir, folder)

        if os.path.isdir(current_input_subfolder):
            resize_image_in_folder(current_input_subfolder, current_output_subfolder, size=size)
        else:
            print(f"Skipping {current_input_subfolder} as it is not a directory.")

    print("Batch resizing completed.")

In [3]:
input_dir = r'D:\Github\CS610_AML_Group_Project\prediction_test\test_image\orginal_data'
output_dir = r'D:\Github\CS610_AML_Group_Project\prediction_test\test_image\resized_data'
batch_resize_images(input_dir, output_dir, size=(128, 128))

100%|██████████| 1/1 [00:00<00:00, 25.71it/s]

Batch resizing completed.


In [4]:
# #Process image data for feature extraction using CNN
input_dir = r'D:\Github\CS610_AML_Group_Project\prediction_test\test_image\resized_data'
img_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])]) #mean and std based on ImageNet - normalise image data closer to normal distribution
img_dataset = datasets.ImageFolder(input_dir, transform=img_transform)
data_loader = DataLoader(img_dataset, batch_size=32, num_workers=4)

In [50]:
#define function for CNN feature extraction
def cnn_feature_extract(cnn_feature_extractor, data_loader):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    #prepare cnn model to use for feature extraction
    cnn_feature_extractor.eval()
    cnn_feature_extractor.fc = torch.nn.Identity() #replace fully connected layer of pretrained cnn with Identity layer
    for para in cnn_feature_extractor.parameters():
        para.requires_grad = False #freeze weights
    #feature extraction
    features_list, labels_list = [], []
    cnn_feature_extractor.to(device)
    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            feature = cnn_feature_extractor(images)
            feature = feature.view(feature.size(0),-1) #flatten into (n_samples, n_features) for non-CNN models
            #convert tensors into numpy for fitting into non-CNN models and add into lists
            features_list.append(feature.cpu().numpy())
            labels_list.append(labels.numpy())

    return cnn_feature_extractor, np.vstack(features_list), np.hstack(labels_list)

In [51]:
class_names = [
        'adidas_forum_high', 'adidas_forum_low', 'adidas_gazelle', 'adidas_nmd_r1',
        'adidas_samba', 'adidas_stan_smith', 'adidas_superstar', 'adidas_ultraboost',
        'asics_gel-lyte_iii', 'converse_chuck_70_high', 'converse_chuck_70_low',
        'converse_chuck_taylor_all-star_high', 'converse_chuck_taylor_all-star_low',
        'converse_one_star', 'new_balance_327', 'new_balance_550', 'new_balance_574',
        'new_balance_990', 'new_balance_992', 'nike_air_force_1_high',
        'nike_air_force_1_low', 'nike_air_force_1_mid', 'nike_air_jordan_1_high',
        'nike_air_jordan_1_low', 'nike_air_jordan_11', 'nike_air_jordan_3',
        'nike_air_jordan_4', 'nike_air_max_1', 'nike_air_max_270', 'nike_air_max_90',
        'nike_air_max_95', 'nike_air_max_97', 'nike_air_max_plus_(tn)',
        'nike_air_vapormax_flyknit', 'nike_air_vapormax_plus', 'nike_blazer_mid_77',
        'nike_cortez', 'nike_dunk_high', 'nike_dunk_low', 'puma_suede_classic',
        'reebok_classic_leather', 'reebok_club_c_85', 'salomon_xt-6',
        'vans_authentic', 'vans_old_skool', 'vans_sk8-hi', 'vans_slip-on_checkerboard',
        'yeezy_700_wave_runner', 'yeezy_boost_350_v2', 'yeezy_slide'
    ]

In [52]:
weights = models.ResNet50_Weights.IMAGENET1K_V2
resnet50_extractor = models.resnet50(weights=weights)
resnet50_extractor, X, _ = cnn_feature_extract(resnet50_extractor, data_loader) #X = features, y =labels

In [53]:
# load model
import pickle
with open('D:/Github/CS610_AML_Group_Project/model_bank/best_cnn_knn_model.pkl', 'rb') as f:
    best_cnn_knn_model = pickle.load(f)

# load data

# predict
predictions = best_cnn_knn_model.predict(X)

print(class_names[predictions[0]])


nike_air_force_1_low


In [54]:
print(y)

[0]
